# LangChain: The LLM Application Framework

LangChain, an open-source library, empowers developers by providing a standardized and structured interface for building and integrating various components of an LLM Application. Its model-agnostic nature allows for compatibility with models from multiple LLM providers, including OpenAI, HuggingFace, and others. 

Using Langchain allows us to build ("like a chain") reusable components as part of complex multi-step LLM-based applications clearly and succinctly. 

You can learn about different [LangChain components here.](https://python.langchain.com/v0.2/docs/concepts/#components)

This tutorial will focus on a few LangChain components and learn about `chaining`, one of its powerful features.

## Prompt templates

[Prompt Templates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) provides templates for designing prompts fed as inputs to the LLM models.
It helps us design templates with multiple inputs that are parameterized and reusable.

```{note}
For this tutorial, we will only cover the use of `String PromptTemplates` as this gives us a finer control over the template string structure, unlike the `ChatPromptTemplate`.
```

Below is an example of how to use a prompt template. We can import this class from the `langchain-core` package.

The `langchain-core` package contains base abstractions of different components and ways to compose them together.

In [1]:
from langchain_core.prompts import PromptTemplate

### String PromptTemplates

The [String PromptTemplates](https://python.langchain.com/v0.2/docs/concepts/#string-prompttemplates) is used to format a string input. By default, the template takes Python `f-string` format. There are currently 2 choices of `template_format` available: `f-string` and `jinja2`. Later we will see the use of `jinja2` format. In the example below, we will use the `f-string` format.

In [2]:
prompt_template = PromptTemplate.from_template(
    "{planet_name} in the solar system is the "
)

prompt_template.format(planet_name="Mars")

'Mars in the solar system is the '

Let's instantiate our OLMo model like in the [previous section](./2-llms-and-prompt-engineering-with-olmo.ipynb#introduction) of the tutorial with `llama-cpp-python`.

In [3]:
from llama_cpp import Llama
from ssec_tutorials import download_olmo_model
from ssec_tutorials.scipy_conf import parse_text_generation_response

In [4]:
OLMO_MODEL = (
    download_olmo_model()
)  # It won't actually download again if it's already there
olmo = Llama(model_path=str(OLMO_MODEL), verbose=False)

Model already exists at /Users/lsetiawan/.cache/ssec_tutorials/OLMo-7B-Instruct-Q4_K_M.gguf


Now that we have our model ready to go, let's try different prompt templating starting from the previous prompt template with an input of `planet_name`.

In [5]:
model_response = olmo(
    prompt=prompt_template.format(planet_name="Mars"),
    temperature=0.2,
    max_tokens=8,
    echo=True,
)  # Generate a completion, can also call olmo.create_completion

In [6]:
print(parse_text_generation_response(model_response))

Mars in the solar system is the 
4th planet from the sun.


In [7]:
# Another example
prompt_template = PromptTemplate.from_template(
    "{entity_1} of the planet {entity_2} is "
)
prompt_template.format(entity_1="Size", entity_2="Earth")

'Size of the planet Earth is '

In [8]:
model_response = olmo(
    prompt=prompt_template.format(entity_1="Size", entity_2="Earth"),
    temperature=0.2,
    echo=True,
)

In [9]:
print(parse_text_generation_response(model_response))

Size of the planet Earth is 
about 12,742 km in diameter. The diameter of the orbit of


#### Your turn 😎

Create a `StringPromptTemplate` that outputs some text generation prompt, for example, "Sun is part of galaxy ...".

Feel free to experiment with the built in [Python `f-string` ](https://docs.python.org/3.11/tutorial/inputoutput.html#formatted-string-literals) for the `prompt` input argument to the model.

In [10]:
# Write your prompt_template and model_response code here

## LLM Interface

LangChain provides us with a standardized interface for loading the LLM model. Once standardized, we can use the same methods across models from different providers to call/invoke functions, enabling reusability.

Loading the model via [LangChain's LlamaCpp](https://python.langchain.com/v0.2/docs/integrations/llms/llamacpp/) abstraction enables us to use the `chaining` feature. This class is part of the `langchain-community` package, which contains third party integrations that are maintained by the LangChain community.

In [11]:
from langchain_community.llms import LlamaCpp

In [12]:
olmo = LlamaCpp(
    model_path=str(OLMO_MODEL),
    temperature=0.8,
    verbose=False,
)

As you can see below, we now have a `LlamaCpp` Langchain object rather than the `Llama` llama-cpp-python object from previous sections.

In [13]:
type(olmo)

langchain_community.llms.llamacpp.LlamaCpp

If you'd like to access the base object `Llama` object from the `llama-cpp-python` package, you can access it via the `.client` attribute of the `LlamaCpp` object.

In [14]:
type(olmo.client)

llama_cpp.llama.Llama

With access to the underlying `Llama` object, you can directly retrieve any metadata information. In this example, we are retrieving OLMo's tokenizer chat template we saw in the [previous notebook](./2-llms-and-prompt-engineering-with-olmo.ipynb#chat-completion) to setup a String PromptTemplate.

The built in model's chat template is using [`jinja2`](https://jinja.palletsprojects.com/en/3.1.x/) templating syntax, which is a popular templating engine for Python.

In [15]:
prompt_template = PromptTemplate.from_template(
    template=olmo.client.metadata["tokenizer.chat_template"], template_format="jinja2"
)

The `PromptTemplate` object has 2 main attributes that are very useful to explore the built-in prompt template of the model:
- `input_variables`: This is a list of all the input variables that the prompt template expects.
- `template`: This is the actual template string that the model uses.

In [16]:
prompt_template.input_variables

['add_generation_prompt', 'eos_token', 'messages']

For this particular template, we can see that it expects `add_generation_prompt`, `eos_token` and `messages`. But what are the variable types for these inputs? What do they mean?

We can answer the questions above by looking at the template string itself. The template string is using the jinja2 templating engine syntax, so it may look confusing at first, but at the end of the day it's essentially just some python code in a template string.

In [17]:
print(prompt_template.template)

{{ eos_token }}{% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<|user|>
' + message['content'] }}
{% elif message['role'] == 'assistant' %}
{{ '<|assistant|>
'  + message['content'] + eos_token }}
{% endif %}
{% if loop.last and add_generation_prompt %}
{{ '<|assistant|>' }}
{% endif %}
{% endfor %}


As we can see above, the template reads as follows:
- `eos_token` is a string that is added at the top of the resulting string after prompt is formatted.
You can also see that `eos_token` is used to append `content` string values from an `assistant` `role`.
You can find this value by going to the Model's [`tokenizer_config.json`](https://huggingface.co/allenai/OLMo-7B-Instruct-hf/blob/main/tokenizer_config.json#L233) file and looking for the `eos_token` key. *Unfornately, this is currently the only way to get this information, you can go to https://github.com/ggerganov/llama.cpp/issues/5040 for more details.* In our case, the `eos_token` is `<|endoftext|>`.
- `messages` is a list of dictionary that is iterated over. As you can see that this dictionary should contain a `role` and `content` key.
- `add_generation_prompt` is a boolean that is used to determine whether to add a generation prompt or not. In this case, when it's the last message and `add_generation_prompt` is `True`, it will add `<|assistant|>` string to the end of the prompt.

Now that we know what the template expects we can create the final prompt string by passing in the expected input variables.

In [18]:
prompt_template.format(
    messages=[
        {
            "role": "user",
            "content": "You are a helpful assistant. Tell me a joke about cats",
        }
    ],
    add_generation_prompt=True,
    eos_token="<|endoftext|>",
)

'<|endoftext|>\n\n<|user|>\nYou are a helpful assistant. Tell me a joke about cats\n\n\n<|assistant|>\n\n'

The output string above contains the necessary signifier tokens for the OLMo Model to understand what the user input is and where the model should put generated responses. This whole string output will then become the full prompt for the model.

## Chain in LangChain

Chaining allows us to combine multiple components, as described above, in series or parallel to develop a multi-step LLM pipeline.
As shown in the image below, any number of components can be linked together to form a chain.

![LancChain Chain](../../images/langchain-chain.webp)


Image Source: [www.analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2023/10/a-comprehensive-guide-to-using-chains-in-langchain/)

Internally, the chain works like below:

STEP 1: Dictionary is processed as an input to the prompt template.  
STEP 2: Prompt Template reads the variables to form the prompt text as output - "What are stars and moon?"  
STEP 3: The prompt is given as input to the LLM model.  
STEP 4: LLM Model produces output.  
STEP 5: The output goes through StrOutputParser that parses it into string and gives the result.  

We can use the pipe operator ("|"), which is part of the [LCEL(Lang Chain Expression Language)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel). The pipe operator sequentially arranges each component, similar to the above image.

In [19]:
llm_chain = prompt_template | olmo

In [20]:
# Construct the prompt as expected by OLMo
llm_chain.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "You are a helpful assistant. Tell me a joke about cats",
            }
        ],
        "add_generation_prompt": True,
        "eos_token": "<|endoftext|>",
    }
)

" Why did the cat like to listen to jazz music? Because it meows a lot, but in a good way! 😉\n\nI hope this silly joke made you smile! Do you want to hear another one? ❓ If not, no worries — I'm here to help with whatever information or assistance you need. 😊"

Instead of having to invoke `llm_chain` repeatedly with `add_generation_prompt` and `eos_token`, we can update our `prompt_template`.

In [21]:
# Create a prompt template using OLMo's tokenizer chat template we saw in module 1, but this time use partial variables.
prompt_template = PromptTemplate.from_template(
    template=olmo.client.metadata["tokenizer.chat_template"],
    template_format="jinja2",
    partial_variables={"add_generation_prompt": True, "eos_token": "<|endoftext|>"},
)

In [22]:
llm_chain = prompt_template | olmo

Let's stream the output instead of waiting for OLMo to generate and display the text. We can use [Callbacks](https://python.langchain.com/v0.2/docs/concepts/#callbacks) to subscribe to various events in your LLM application pipeline. Check [this out](https://python.langchain.com/v0.1/docs/modules/callbacks/#callback-handlers) for a list of events. 

In [23]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler

In [24]:
llm_chain.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "You are a helpful assistant. Tell me a joke about cats",
            }
        ]
    },
    config={"callbacks": [StreamingStdOutCallbackHandler()]},
)

 Why did the cat like to listen to music on its telephone? Because it made a cute phoneoony when you called! (Did I do that right?) Cats and phones have been making the sound of the tunee (that's an up down beep sounds combined into one, for those not in the know!) since before the invention of the telephone, so this one might be a bit old-school. But it's still purrfectly fitting! 😻🐈📱🎉💬

" Why did the cat like to listen to music on its telephone? Because it made a cute phoneoony when you called! (Did I do that right?) Cats and phones have been making the sound of the tunee (that's an up down beep sounds combined into one, for those not in the know!) since before the invention of the telephone, so this one might be a bit old-school. But it's still purrfectly fitting! 😻🐈📱🎉💬"

We will cover more LangChain concepts in upcoming notebooks. 

#### Your turn 😎

Try different messages value(s) and see how the output changes. But remember to follow the template structure.
The dictionary keys must contain `role` and `content` and the allowed `role` values are only `user` and `assistant`.

In [25]:
# Write your llm_chain.invoke code here, feel free to also, create your own template and try partial_variables